In [12]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras import backend as K
import src.data_augmentation as da
import src.pre_processing as pp
K.set_image_dim_ordering('th')

In [14]:
training = pp.get_training()
test = pp.get_test()

In [ ]:
X_train = np.array([x for x, y in training])
y_train = np.array([y for x, y in training])
X_test = test
X_train = X_train.reshape(X_train.shape[0], 1, 64, 64)
datagen = da.get_mnist_image_generator(X_train)
X_train = X_train.reshape(-1, 64, 64)
y_train = y_train.reshape(-1, 1)

In [15]:
pixel_count = X_train.shape[1] * X_train.shape[2]

In [16]:
# Normalize values (0-1)
# X_train = np.divide(X_train, 255)
# X_test = np.divide(X_test, 255)
y_train = np_utils.to_categorical(y_train)

In [17]:
class_count = y_train.shape[1]

In [18]:
# Seed for reproducibility
seed = 42
np.random.seed(seed)

In [20]:
# FOR CNN
X_train = X_train.reshape(X_train.shape[0], 1, 64, 64).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 64, 64).astype('float32')

In [16]:
def base_cnn():
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(1, 64, 64), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(class_count, activation='softmax'))
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [32]:
def larger_cnn():
    # create model
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(1, 64, 64), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dense(50, activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dense(class_count, activation='softmax'))
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [23]:
def deep_cnn():
    # create model
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='linear', input_shape=(1, 64, 64)))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(64, (5, 5), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(1024, activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dense(1024, activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dense(512, activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dense(class_count, activation='softmax'))
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
cnn_model_deep = deep_cnn()
cnn_model_deep.fit(X_train, y_train, batch_size=128, epochs=500, verbose=2)

In [56]:
cnn_deep_pred = cnn_model_deep.predict(X_test)

In [57]:
cnn_deep_pred_nums = np.array([np.argmax(pred) for pred in cnn_deep_pred])

In [58]:
cnn_deep_pred_final = np.array([(i, x) for (i, x) in enumerate(cnn_deep_pred_nums)])

In [59]:
np.savetxt('../data/test_y_deep_cnn.csv', cnn_deep_pred_final, delimiter=',', fmt='%d', header='Id,Label')